<a href="https://colab.research.google.com/github/Radperia/SIGNATE_student_2020/blob/master/signate_dogsub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 / 60 / 60 "h)"}'

0.161704days (3.8809h)


In [24]:
!git clone https://github.com/microsoft/LightGBM

fatal: destination path 'LightGBM' already exists and is not an empty directory.


In [25]:
!cd LightGBM && mkdir build && cd build && cmake .. && make -j"$(nproc)"

mkdir: cannot create directory ‘build’: File exists


In [26]:
!cd LightGBM/python-package && python setup.py install --precompile

running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
running install_lib
INFO:LightGBM:Installing lib_lightgbm from: ['../lib_lightgbm.so']
running install_egg_info
removing '/usr/local/lib/python3.6/dist-packages/lightgbm-3.0.0-py3.6.egg-info' (and everything under it)
Copying lightgbm.egg-info to /usr/local/lib/python3.6/dist-packages/lightgbm-3.0.0-py3.6.egg-info
running install_scripts


In [27]:
import numpy as np
import pandas as pd
from contextlib import contextmanager
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import time
import re
import string
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import gc
from collections import defaultdict
import os
import psutil
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [28]:
train_df = pd.read_csv('./drive/My Drive/SIGNATE/train.csv').fillna(' ')
test_df  = pd.read_csv('./drive/My Drive/SIGNATE/test.csv').fillna(' ')
sample = pd.read_csv('./drive/My Drive/SIGNATE/submit_sample.csv', header=None)

train_text = train_df['description']
test_text = test_df['description']
all_text = pd.concat([train_text, test_text])

sentences = train_df['description'].tolist() + test_df['description'].tolist()
sentences[::500]  # 500個飛ばしに確認

['Executes and writes portions of testing plans, protocols, and documentation for assigned portion of application; identifies and debugs issues with code and suggests changes or improvements.',
 'Knowledge and experience of complex software design for distributed systems in embedded networking/telecommunications projects.',
 'Analyze client business processes of storing and managing data.',
 'Defining tasks, timeline and required resources to deliver on the mechanical design portion of a project brief, with strong accountability for successful completion of agreed upon deliverables.',
 'Familiar with sensors, transducers, physiologic modeling, and transfer functions',
 'Manage, maintain, refresh and upgrade environmental components (including patching, etc.)',
 'Conduct research on cutting-edge techniques and tools in machine learning/deep learning/artificial intelligence',
 'Assist in planning and development of visualizations and analytic aggregations to best portray discovered insig

tfidfの文献があったからやってみる https://www.kaggle.com/ogrellier/lgbm-with-words-and-chars-n-gram

ここから　https://www.kaggle.com/peterhurford/lightgbm-with-select-k-best-on-tfidf


In [29]:
import lightgbm as lgb
from sklearn import metrics

calc_f1 = lambda y, p: metrics.f1_score(y, p.argmax(axis=1), average='macro')

def macro_f1(pred: np.array, data: lgb.Dataset):
    y = data.get_label()
    pred = pred.reshape(-1, len(y)).T  # -> (N, num_class)

    f1 = calc_f1(y, pred)
    return 'macro_f1', f1, True  # True means "higher is better"

In [30]:
my_params={
    'objective': 'multiclass',
    'metric': 'custom',
    'num_class': 4,
    'num_iterations': 2000,
    'learning_rate': 0.0001,
    'max_depth': -1,
    'num_leaves': 15,
    'max_bin': 40,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'nthread': -1,
    'bagging_freq': 1,
    'verbose': -1,
    'seed': 42,
}

In [31]:
dog_params = {
    'objective': 'multiclass',
    'metric': 'custom',
    'num_class': 4,
    'learning_rate': 0.01,
    'max_depth': -1,
    'num_leaves': 15,
    'max_bin': 31,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'nthread': -1,
    'bagging_freq': 1,
    'verbose': -1,
    'seed': 1,
    }

In [32]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000)  # 出現頻度上位{num_words}だけを用いる
tokenizer.fit_on_texts(sentences)
print(list(tokenizer.word_index)[:10] ) # 学習された辞書（出現頻度順）

['and', 'to', 'the', 'of', 'with', 'data', 'in', 'for', 'a', 'business']


In [33]:
# 学習・検証に分けて，ベクトルを生成。加えて，正解ラベルも作成
train_X, test_X = np.split(tokenizer.texts_to_matrix(sentences, mode='binary'),
                           [len(train_df)], axis=0)

train_y = train_df['jobflag'].values - 1  # maps {1, 2, 3 ,4} -> {0, 1, 2, 3}
train_X.shape, train_y.shape, test_X.shape

((2931, 1000), (2931,), (1743, 1000))

In [34]:
weight = 1 / pd.DataFrame(train_y).reset_index().groupby(0).count().values
weight = weight[train_y].ravel()
weight /= weight.sum()

print(weight)

dtrain = lgb.Dataset(train_X, train_y, weight=weight)

[0.00071839 0.00018169 0.00042882 ... 0.00040064 0.00018169 0.00071839]


In [35]:
import collections
print(collections.Counter(train_df['jobflag']))

Counter({3: 1376, 1: 624, 4: 583, 2: 348})


SVCでの最適化、class_weightでクラス間の重み変えてるつもりだけどどうも違うっぽい

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import datasets
from sklearn.svm import SVC
def param():
  tuned_parameters = {
      'C': [0.1, 1, 10],
      'kernel':['rbf', 'linear', 'poly'],
      'gamma': ['auto'],
      'class_weight': ['balanced']
  }
  return tuned_parameters

score = 'f1'

svc_cv = GridSearchCV(SVC(), param(), n_jobs=-1, cv=5, verbose=3, scoring='%s_weighted'%score)
'''
svc_cv.fit(train_X, train_y)

svc_result = pd.DataFrame.from_dict(svc_cv.cv_results_)
svc_result.to_csv('svc_result.csv')
'''

"\nsvc_cv.fit(train_X, train_y)\n\nsvc_result = pd.DataFrame.from_dict(svc_cv.cv_results_)\nsvc_result.to_csv('svc_result.csv')\n"

In [37]:
#best = svc_cv.best_estimator_
#pred = best.predict(test_X)

In [38]:
print(pred.shape)

NameError: ignored

In [ ]:
pred = pred + 1

In [ ]:
sub = sample
sub[1] = pred

In [ ]:
from google.colab import files
pd.DataFrame(sub, index=test_df.index).to_csv('adjusted_svc.csv', header=None)
files.download('adjusted_svc.csv')

SVCやったし、RFCもやりたい

In [39]:
# nearly 6 hours
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

rfc_param = {
    'n_estimators': [10, 50, 100],
    'max_depth': [10, 30, 50],
    'min_samples_leaf': [2, 4, 6],
    'min_samples_split': [2, 5],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': ['True'],
    'class_weight': ['balanced', 'balanced_subsample']
}

rfc = RandomForestClassifier()
rfc_cv = GridSearchCV(estimator=rfc, param_grid=rfc_param, cv=5, verbose=3, n_jobs=-1, scoring='%s_weighted'%'f1')

流石に通り数が多すぎた 13500で450minかかる  
750/45 で0.39台

In [40]:
from sklearn.ensemble import StackingClassifier

estimators = [
              ('rfc', rfc_cv),
              ('svc', svc_cv)
]

clf_stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=1000))
clf_stack.fit(train_X, train_y)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  6.1min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  5.2min finished


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


...

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  3.3min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  3.3min finished


StackingClassifier(cv=None,
                   estimators=[('rfc',
                                GridSearchCV(cv=5, error_score=nan,
                                             estimator=RandomForestClassifier(bootstrap=True,
                                                                              ccp_alpha=0.0,
                                                                              class_weight=None,
                                                                              criterion='gini',
                                                                              max_depth=None,
                                                                              max_features='auto',
                                                                              max_leaf_nodes=None,
                                                                              max_samples=None,
                                                                              min_impurity_decrea

In [43]:
clf_stack.get_params()

{'cv': None, 'estimators': [('rfc', GridSearchCV(cv=5, error_score=nan,
                estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini', max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,...
                                       

In [41]:
clf_stack.score(train_X, train_y)

0.7932446264073695

In [42]:
stacking_pred = clf_stack.predict(test_X)
stacking_pred = stacking_pred + 1
stacking_sub = sample
stacking_sub[1] = stacking_pred

from google.colab import files
pd.DataFrame(stacking_sub, index=test_df.index).to_csv('stacking_clf_0.7932.csv', header=None)
files.download('stacking_clf_0.7932.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cvbooster = lgb.cv(my_params, dtrain, return_cvbooster=True, stratified=False, 
        num_boost_round=9999, verbose_eval=100, early_stopping_rounds=500,
        feval=macro_f1)['cvbooster']

In [ ]:
train_df = pd.read_csv('./drive/My Drive/SIGNATE/train.csv', index_col=0)
test_df  = pd.read_csv('./drive/My Drive/SIGNATE/test.csv',  index_col=0)

In [ ]:
pred = np.stack(cvbooster.predict(test_X)).mean(axis=0).argmax(axis=1) + 1
pd.DataFrame(pred, index=test_df.index).to_csv('./myversion_dog_lgb814.csv', header=None)

In [ ]:
from google.colab import files
files.download('myversion_dog_lgb814.csv')